In [ ]:
from ca_lecture_hall_model import Grid

In [ ]:
# Generate a grid for given size, density and spread threshold
size = 20
density = 0.7
spread_threshold = 0.3

g = Grid(size, density, spread_threshold)

In [ ]:
# Initialize the board and set the initial spreader
g.initialize_board()
g.set_spreader(13, 7)
g.show_grid()

In [ ]:
# Run one time step (ru-run this cell as needed for testing)
g.update_grid()
g.show_grid()